In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path



In [8]:
cwd =  Path.cwd()
cwd = cwd.parents[1]
folder = cwd /'test'/'IBActivityNew'

tables = cwd /'test'/ 'Tables'
tables

WindowsPath('d:/Javier/SynologyDrive/Portfolio Python/test/Tables')

In [21]:
def process_fees(data):
    data_ca = data.loc[data.a =='Fees']
    if len(data_ca) > 0:
        ca = data_ca.loc[data_ca.b == 'Data']
        if len(ca) > 0:
            first_row = ca.index.values[0]-1
            header_row = list(data_ca.loc[first_row])
            ca.columns = header_row
            ca = ca.loc[:,ca.columns.notna()] 
            ca.drop(columns=['Fees','Header'], inplace=True, errors='ignore')
            ca.dropna(axis=0,inplace=True)
            return ca

In [4]:
def convert_to_df(a_file):
    data = pd.read_csv(folder/a_file, names=list('abcdefghijklmnopq'))
    #data.drop(data.loc[(data.a == 'Notes/Legal Notes') | (data.a == 'Codes') ].index, inplace= True)
    #data = (data.a != 'Codes') & (data.a != 'Notes/Legal Notes')
    data = data.loc[~data.a.isin(['Statement','Codes', 'Notes/Legal Notes','Account Information','Change in NAV','Mark-to-Market Performance Summary','Realized & Unrealized Performance Summary','Open Positions'])]
    return data

In [17]:
def get_all_fees(folder):
    #Creates a list of all files in folder
    filelist = os.listdir(folder)
    #Creates a empty list to append all dataframes
    fees_list = []
    #Iterates through all files in filelist
    for file_ in filelist:
        #Checks if file is a .csv
        if file_.lower().endswith('.csv'):
            data = convert_to_df(file_)
            fee = process_fees(data)
            if type(fee) != 'NonType':
                fees_list.append(fee)
    fees = pd.concat(fees_list,sort=False )
    return fees

In [22]:
%%time
fees = get_all_fees(folder)
fees

Wall time: 4.43 s


,Subtitle,Currency,Date,Description,Amount
114,Other Fees,USD,2017-04-04,AMEX Level I for Mar 2017,-1.5
115,Other Fees,USD,2017-04-04,OPRA (US Options Exchanges) Non-Professional L...,-1.5
129,Other Fees,USD,2017-05-03,AMEX Level I for Apr 2017,-1.5
130,Other Fees,USD,2017-05-03,BME (MEFF) Non-Professional Level I for Apr 2017,-2.45
131,Other Fees,USD,2017-05-03,NYSE Level I for Apr 2017,-1.5
...,...,...,...,...,...
1017,Other Fees,USD,2019-10-02,j******17:US Equity and Options Add-On Streami...,-4.5
1018,Other Fees,USD,2019-10-02,J******17:EURVALUENP FOR OCT 2019,-13.62
1019,Other Fees,USD,2019-10-02,J******17:HKFE L1 FOR OCT 2019,-3.19
1020,Other Fees,USD,2019-10-02,j******17:US Securities Snapshot and Futures V...,-10
